In [1]:
pip install clean-fid


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os, sys, socket, platform
print("Python:", sys.executable)
print("Host:", socket.gethostname(), "| OS:", platform.platform())
print("CWD:", os.getcwd())


Python: /home/group-3/miniconda3/bin/python
Host: lyskamm-1-1xl40s-4 | OS: Linux-6.8.0-86-generic-x86_64-with-glibc2.39
CWD: /home/group-3/tests_L/MixDQ/scripts


In [18]:
import os, glob

def count_recursive(root):
    root = os.path.abspath(os.path.expanduser(root))
    exts = ('.png','.jpg','.jpeg','.bmp','.webp')
    files = [p for p in glob.iglob(os.path.join(root, '**', '*'), recursive=True)
             if p.lower().endswith(exts)]
    print(f"{root}: {len(files)} images")
    print("sample:", files[:3])
    return root, files

# CHANGE REAL to your actual COCO val2014 images folder (not the repo's scripts/utils/)
REAL = "/home/group-3/tests_L/MixDQ/scripts/utils/val2014"  # e.g. /data/coco/val2014
FP16 = "/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images"
MIXD = "/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big"

real_root, real_list = count_recursive(REAL)
fp_root,   fp_list   = count_recursive(FP16)
mix_root,  mix_list  = count_recursive(MIXD)


/home/group-3/tests_L/MixDQ/scripts/utils/val2014: 40504 images
sample: ['/home/group-3/tests_L/MixDQ/scripts/utils/val2014/COCO_val2014_000000407960.jpg', '/home/group-3/tests_L/MixDQ/scripts/utils/val2014/COCO_val2014_000000577868.jpg', '/home/group-3/tests_L/MixDQ/scripts/utils/val2014/COCO_val2014_000000573395.jpg']
/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images: 1024 images
sample: ['/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images/878.png', '/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images/349.png', '/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images/268.png']
/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big: 1024 images
sample: ['/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big/878.png', '/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big/349.png', '/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big/268.png']


In [ ]:
import os, pathlib, shutil
from cleanfid import fid

def flatten_symlink(files, out_dir):
    out = pathlib.Path(out_dir).resolve()
    out.mkdir(parents=True, exist_ok=True)
    for i, src in enumerate(files):
        ext = pathlib.Path(src).suffix.lower() or ".png"
        dst = out / f"{i:08d}{ext}"
        try:
            os.symlink(os.path.abspath(src), dst)   # fast, no copy
        except (OSError, AttributeError):           # fallback if symlinks not allowed
            shutil.copy2(src, dst)
    return str(out)

flat_real = flatten_symlink(real_list, "/tmp/fid_flat_real")
flat_fp16 = flatten_symlink(fp_list,   "/tmp/fid_flat_fp16")
flat_mixd = flatten_symlink(mix_list,  "/tmp/fid_flat_mixd")

print("Flat dirs:", flat_real, flat_fp16, flat_mixd)

# Option A: folder vs folder
fid_fp16  = fid.compute_fid(flat_real, flat_fp16, mode="clean", num_workers=0, verbose=True)
fid_mixdq = fid.compute_fid(flat_real, flat_mixd, mode="clean", num_workers=0, verbose=True)
print("FID FP16:", fid_fp16, " | FID MixDQ:", fid_mixdq)


Flat dirs: /tmp/fid_flat_real /tmp/fid_flat_fp16 /tmp/fid_flat_mixd
compute FID between two folders
Found 40504 images in the folder /tmp/fid_flat_real


FID fid_flat_real : 100%|██████████| 1266/1266 [07:46<00:00,  2.72it/s]


Found 1024 images in the folder /tmp/fid_flat_fp16


FID fid_flat_fp16 : 100%|██████████| 32/32 [00:16<00:00,  1.93it/s]


compute FID between two folders
Found 40504 images in the folder /tmp/fid_flat_real


...

saving custom FID stats to /home/group-3/miniconda3/lib/python3.13/site-packages/cleanfid/stats/coco2014_val_flat_clean_custom_na.npz
saving custom KID stats to /home/group-3/miniconda3/lib/python3.13/site-packages/cleanfid/stats/coco2014_val_flat_clean_custom_na_kid.npz
compute FID of a folder with coco2014_val_flat statistics
Found 1024 images in the folder /tmp/fid_flat_fp16


FID fid_flat_fp16 : 100%|██████████| 32/32 [00:25<00:00,  1.28it/s]


compute FID of a folder with coco2014_val_flat statistics
Found 64 images in the folder /tmp/fid_flat_mixd


FID fid_flat_mixd : 100%|██████████| 2/2 [00:03<00:00,  1.69s/it]


FID (custom stats) FP16: 85.81415229915007  | MixDQ: 232.16348995764372


In [12]:
import json

COCO_JSON = "/home/group-3/tests_L/MixDQ/scripts/utils/captions_val2014.json"
OUT_TXT   = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts.txt"

with open(COCO_JSON, "r", encoding="utf-8") as f:
    coco = json.load(f)

# map image_id -> first caption
caps_by_id = {}
for ann in coco["annotations"]:
    caps_by_id.setdefault(ann["image_id"], ann["caption"].strip())

# order images by file_name (lexicographic), then take the first caption for each
images_sorted = sorted(coco["images"], key=lambda x: x["file_name"])
prompts = [caps_by_id.get(img["id"], "a photo") for img in images_sorted]

with open(OUT_TXT, "w", encoding="utf-8") as f:
    f.write("\n".join(prompts))
print("Wrote:", OUT_TXT)


Wrote: /home/group-3/tests_L/MixDQ/scripts/utils/prompts.txt


In [19]:
# === ONE-CELL: FID + CLIPScore + RI@1 =========================================
# Fill these with ABSOLUTE paths on your VM:
REAL_DIR   = "/home/group-3/tests_L/MixDQ/scripts/utils/val2014"                    # real images (COCO val2014, or your reference set)
FP16_DIR   = "/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images"
MIXDQ_DIR  = "/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big"
PROMPTS_TXT = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts.txt"                      # one prompt per image, same order as the generated images

# ------------------------------------------------------------------------------
import os, sys, glob, math, tempfile, shutil, pathlib
from typing import List, Tuple
import numpy as np
from PIL import Image

# Torch / CLIP
import torch, torch.nn.functional as F
import open_clip
from tqdm import tqdm

# Clean-FID
from cleanfid import fid

# ---- helpers -----------------------------------------------------------------
def _abs(p): return os.path.abspath(os.path.expanduser(p))

def list_images_recursive(root: str, exts=(".png",".jpg",".jpeg",".bmp",".webp")) -> List[str]:
    root = _abs(root)
    return [p for p in glob.iglob(os.path.join(root, "**", "*"), recursive=True)
            if os.path.isfile(p) and p.lower().endswith(exts)]

def flatten_to_tmp(files: List[str], tmp_label: str) -> str:
    out = pathlib.Path(tempfile.mkdtemp(prefix=f"fid_flat_{tmp_label}_")).resolve()
    for i, src in enumerate(files):
        ext = pathlib.Path(src).suffix.lower() or ".png"
        dst = out / f"{i:08d}{ext}"
        try:
            os.symlink(_abs(src), dst)  # fast if fs allows
        except Exception:
            shutil.copy2(_abs(src), dst)
    return str(out)

def read_prompts(prompts_path: str) -> List[str]:
    with open(_abs(prompts_path), "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    if not lines:
        raise ValueError("No prompts found in PROMPTS_TXT.")
    return lines

def natural_key(s: str):
    import re
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", s)]

@torch.no_grad()
def embed_images(files: List[str], model, preprocess, device, batch=64) -> torch.Tensor:
    feats = []
    for i in tqdm(range(0, len(files), batch), desc="embed images"):
        ims = []
        for p in files[i:i+batch]:
            try:
                img = Image.open(p).convert("RGB")
            except Exception:
                continue
            ims.append(preprocess(img))
        if not ims:
            continue
        ims = torch.stack(ims).to(device)
        f = model.encode_image(ims)
        feats.append(F.normalize(f.float(), dim=-1))
    if not feats:
        raise ValueError("No image features computed (check image readability).")
    return torch.cat(feats, dim=0)

@torch.no_grad()
def embed_texts(texts: List[str], model, tokenizer, device, batch=256) -> torch.Tensor:
    feats = []
    for i in tqdm(range(0, len(texts), batch), desc="embed texts"):
        toks = tokenizer(texts[i:i+batch]).to(device)
        f = model.encode_text(toks)
        feats.append(F.normalize(f.float(), dim=-1))
    return torch.cat(feats, dim=0)

def clipscore_and_ri1(gen_files: List[str], prompts: List[str], device="cuda") -> Tuple[float, float]:
    # Load CLIP
    model, _, preprocess = open_clip.create_model_and_transforms("ViT-L-14-336", pretrained="openai", device=device)
    tokenizer = open_clip.get_tokenizer("ViT-L-14-336")
    model.eval()

    # Sort for deterministic alignment
    files_sorted = sorted(gen_files, key=natural_key)

    if len(files_sorted) != len(prompts):
        raise ValueError(f"Mismatch: {len(files_sorted)} images vs {len(prompts)} prompts. "
                         f"Ensure PROMPTS_TXT lines match generated images 1:1 (same order as sorted filenames).")

    img_feat = embed_images(files_sorted, model, preprocess, device)
    txt_feat = embed_texts(prompts, model, tokenizer, device)

    # Similarity matrix (text x image), cosine since features are normalized
    S = txt_feat @ img_feat.T
    diag = torch.diag(S)
    clipscore_mean = (torch.clamp(diag, min=0.0) * 2.5).mean().item()  # standard scaling

    # RI@1: does each prompt retrieve its own image as top-1?
    top1 = S.argmax(dim=1)
    ri1 = (top1 == torch.arange(S.size(0), device=top1.device)).float().mean().item()

    return clipscore_mean, ri1

def compute_fid_folder_vs_folder(real_files: List[str], gen_files: List[str], tag: str) -> float:
    flat_real = flatten_to_tmp(real_files, f"real_{tag}")
    flat_gen  = flatten_to_tmp(gen_files,  f"gen_{tag}")
    # Folder-vs-folder FID (non-recursive; we pre-flatten via symlink/copy)
    score = fid.compute_fid(flat_real, flat_gen, mode="clean", num_workers=0, verbose=True)
    # Cleanup note: temp dirs auto-cleaned on reboot; keep for reproducibility this run.
    print(f"[{tag}] temp real dir:", flat_real)
    print(f"[{tag}] temp gen  dir:", flat_gen)
    return score

# ---- run ---------------------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 1) Collect files (recursively), and sort deterministically
real_files = sorted(list_images_recursive(REAL_DIR), key=natural_key)
fp16_files = sorted(list_images_recursive(FP16_DIR), key=natural_key)
mixd_files = sorted(list_images_recursive(MIXDQ_DIR), key=natural_key)

print(f"Found: real={len(real_files)} | fp16={len(fp16_files)} | mixdq={len(mixd_files)}")
if len(real_files) == 0 or len(fp16_files) == 0 or len(mixd_files) == 0:
    raise FileNotFoundError("One or more folders have 0 readable images. Check the ABSOLUTE paths and extensions.")

# 2) Load prompts (one per image, in the SAME order as the sorted filenames above)
prompts = read_prompts(PROMPTS_TXT)

# 3) CLIPScore & RI@1
clip_fp16, ri1_fp16 = clipscore_and_ri1(fp16_files, prompts, device=device)
clip_mixd, ri1_mixd = clipscore_and_ri1(mixd_files, prompts, device=device)

# 4) FID (folder vs folder; Clean-FID "clean" mode)
fid_fp16  = compute_fid_folder_vs_folder(real_files, fp16_files, tag="FP16")
fid_mixdq = compute_fid_folder_vs_folder(real_files, mixd_files,  tag="MixDQ")

# 5) Print summary
print("\n=== SUMMARY ===")
print(f"CLIPScore (mean)  FP16:  {clip_fp16:.4f}    MixDQ: {clip_mixd:.4f}")
print(f"RI@1              FP16:  {ri1_fp16*100:5.2f}%  MixDQ: {ri1_mixd*100:5.2f}%")
print(f"FID (clean)       FP16:  {fid_fp16:.4f}       MixDQ: {fid_mixdq:.4f}")
# ==============================================================================


ModuleNotFoundError: No module named 'open_clip'

In [21]:
# === ONE-CELL: FID + CLIPScore + RI@1 with open_clip OR HF fallback ===========
# >>>>>> EDIT THESE (absolute paths) <<<<<<
REAL_DIR   = "/home/group-3/tests_L/MixDQ/scripts/utils/val2014"                    # real images (COCO val2014, or your reference set)
FP16_DIR   = "/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images"
MIXDQ_DIR  = "/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big"
PROMPTS_TXT = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts.txt"                       # one prompt per image, same order as filenames

# ------------------------------------------------------------------------------
import os, sys, glob, tempfile, shutil, pathlib, math, types
from typing import List, Tuple
import numpy as np
from PIL import Image

import torch, torch.nn.functional as F
from tqdm import tqdm

# Clean-FID
try:
    from cleanfid import fid
except Exception as e:
    raise RuntimeError("clean-fid is required. Install with: pip install clean-fid") from e

def _abs(p): return os.path.abspath(os.path.expanduser(p))

def list_images_recursive(root: str, exts=(".png",".jpg",".jpeg",".bmp",".webp")) -> List[str]:
    root = _abs(root)
    return [p for p in glob.iglob(os.path.join(root, "**", "*"), recursive=True)
            if os.path.isfile(p) and p.lower().endswith(exts)]

def flatten_to_tmp(files: List[str], tmp_label: str) -> str:
    out = pathlib.Path(tempfile.mkdtemp(prefix=f"fid_flat_{tmp_label}_")).resolve()
    for i, src in enumerate(files):
        ext = pathlib.Path(src).suffix.lower() or ".png"
        dst = out / f"{i:08d}{ext}"
        try:
            os.symlink(_abs(src), dst)    # fast, if allowed
        except Exception:
            shutil.copy2(_abs(src), dst)  # fallback: copy
    return str(out)

def read_prompts(prompts_path: str) -> List[str]:
    with open(_abs(prompts_path), "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    if not lines:
        raise ValueError("No prompts found in PROMPTS_TXT.")
    return lines

def natural_key(s: str):
    import re
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", os.path.basename(s))]

# ---- CLIP backends ------------------------------------------------------------
class ClipBackend:
    def __init__(self, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.name = None
        self.model = None
        self.preprocess = None
        self.tokenizer = None
        self.processor = None

        # Try open_clip
        try:
            import open_clip
            self.model, _, self.preprocess = open_clip.create_model_and_transforms(
                "ViT-L-14-336", pretrained="openai", device=device
            )
            self.tokenizer = open_clip.get_tokenizer("ViT-L-14-336")
            self.model.eval()
            self.name = "open_clip"
        except Exception:
            # Fallback: Hugging Face transformers
            try:
                from transformers import CLIPModel, CLIPProcessor
            except Exception as e:
                raise RuntimeError(
                    "Neither open_clip_torch nor transformers (CLIP) is available.\n"
                    "Install one of:\n"
                    "  pip install open-clip-torch\n"
                    "  or\n"
                    "  pip install transformers"
                ) from e
            from transformers import CLIPModel, CLIPProcessor
            self.model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336").to(device)
            self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
            self.model.eval()
            self.name = "hf_clip"

    @torch.no_grad()
    def embed_images(self, files: List[str], batch: int = 64) -> torch.Tensor:
        feats = []
        if self.name == "open_clip":
            for i in tqdm(range(0, len(files), batch), desc="embed images (open_clip)"):
                imgs = []
                for p in files[i:i+batch]:
                    try:
                        imgs.append(self.preprocess(Image.open(p).convert("RGB")))
                    except Exception:
                        continue
                if not imgs: 
                    continue
                ims = torch.stack(imgs).to(self.device)
                f = self.model.encode_image(ims)
                feats.append(F.normalize(f.float(), dim=-1))
        else:  # hf_clip
            for i in tqdm(range(0, len(files), batch), desc="embed images (hf_clip)"):
                imgs = []
                for p in files[i:i+batch]:
                    try:
                        imgs.append(Image.open(p).convert("RGB"))
                    except Exception:
                        continue
                if not imgs:
                    continue
                inputs = self.processor(images=imgs, return_tensors="pt").to(self.device)
                f = self.model.get_image_features(**inputs)
                feats.append(F.normalize(f.float(), dim=-1))
        if not feats:
            raise ValueError("No image features computed (check image readability).")
        return torch.cat(feats, dim=0)

    @torch.no_grad()
    def embed_texts(self, texts: List[str], batch: int = 256) -> torch.Tensor:
        feats = []
        if self.name == "open_clip":
            tok = self.tokenizer
            for i in tqdm(range(0, len(texts), batch), desc="embed texts (open_clip)"):
                t = tok(texts[i:i+batch]).to(self.device)
                f = self.model.encode_text(t)
                feats.append(F.normalize(f.float(), dim=-1))
        else:  # hf_clip
            for i in tqdm(range(0, len(texts), batch), desc="embed texts (hf_clip)"):
                inputs = self.processor(text=texts[i:i+batch], return_tensors="pt", padding=True, truncation=True).to(self.device)
                f = self.model.get_text_features(**inputs)
                feats.append(F.normalize(f.float(), dim=-1))
        return torch.cat(feats, dim=0)

# ---- Metrics ------------------------------------------------------------------
def clipscore_and_ri1(gen_files: List[str], prompts: List[str], backend: ClipBackend) -> Tuple[float, float]:
    files_sorted = sorted(gen_files, key=natural_key)
    if len(files_sorted) != len(prompts):
        raise ValueError(f"Mismatch: {len(files_sorted)} images vs {len(prompts)} prompts.\n"
                         f"Ensure prompts.txt lines match the sorted filenames 1:1.")
    img_feat = backend.embed_images(files_sorted)
    txt_feat = backend.embed_texts(prompts)

    S = txt_feat @ img_feat.T             # cosine since features are normalized
    diag = torch.diag(S)
    clipscore_mean = (torch.clamp(diag, min=0.0) * 2.5).mean().item()  # standard scaling
    top1 = S.argmax(dim=1)
    ri1 = (top1 == torch.arange(S.size(0), device=top1.device)).float().mean().item()
    return clipscore_mean, ri1

def compute_fid_folder_vs_folder(real_files: List[str], gen_files: List[str], tag: str) -> float:
    flat_real = flatten_to_tmp(real_files, f"real_{tag}")
    flat_gen  = flatten_to_tmp(gen_files,  f"gen_{tag}")
    score = fid.compute_fid(flat_real, flat_gen, mode="clean", num_workers=0, verbose=True)
    print(f"[{tag}] temp real dir:", flat_real)
    print(f"[{tag}] temp gen  dir:", flat_gen)
    return score

# ---- Run ----------------------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

real_files = sorted(list_images_recursive(REAL_DIR), key=natural_key)
fp16_files = sorted(list_images_recursive(FP16_DIR), key=natural_key)
mixd_files = sorted(list_images_recursive(MIXDQ_DIR), key=natural_key)

print(f"Found files: real={len(real_files)} | fp16={len(fp16_files)} | mixdq={len(mixd_files)}")
if len(real_files) == 0 or len(fp16_files) == 0 or len(mixd_files) == 0:
    raise FileNotFoundError("One or more folders have 0 readable images. Use absolute paths and ensure top-level images or nested (we handle nested).")

prompts = read_prompts(PROMPTS_TXT)

backend = ClipBackend(device=device)
print("CLIP backend:", backend.name)

clip_fp16, ri1_fp16 = clipscore_and_ri1(fp16_files, prompts, backend)
clip_mixd, ri1_mixd = clipscore_and_ri1(mixd_files, prompts, backend)

fid_fp16  = compute_fid_folder_vs_folder(real_files, fp16_files, tag="FP16")
fid_mixdq = compute_fid_folder_vs_folder(real_files, mixd_files,  tag="MixDQ")

print("\n=== SUMMARY ===")
print(f"CLIPScore (mean)  FP16:  {clip_fp16:.4f}    MixDQ: {clip_mixd:.4f}")
print(f"RI@1              FP16:  {ri1_fp16*100:5.2f}%  MixDQ: {ri1_mixd*100:5.2f}%")
print(f"FID (clean)       FP16:  {fid_fp16:.4f}       MixDQ: {fid_mixdq:.4f}")
# ==============================================================================


Device: cuda
Found files: real=40504 | fp16=1024 | mixdq=1024
CLIP backend: hf_clip


ValueError: Mismatch: 1024 images vs 40504 prompts.
Ensure prompts.txt lines match the sorted filenames 1:1.